In [5]:
import mysql.connector as mysql
from mysql.connector import Error
import sqlite3
import numpy as np
import pandas as pd

### DATA PREPROCESSING

In [6]:
# Nhận tất cả dữ liệu thô từ database

try:
    connection = mysql.connect(host='localhost', database='air_quality', user='root', password='password')
    if connection.is_connected():
        cursor = connection.cursor()
        sql = "select * from air_quality_raw"
        cursor.execute(sql)
        record = cursor.fetchall()
        print('Kích thước dữ liệu: ', len(record))
        connection.commit()
        cursor.close()
except Error as e:
    print('Error', e)

Error Authentication plugin 'caching_sha2_password' is not supported


In [7]:
# Chuyển dữ liệu sang dạng DataFrame
df = pd.DataFrame(record, columns=['ID', 'City', 'Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3',
                                   'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'])
# df = pd.read_csv('IOT/city_day.csv')
df = df.drop(columns=['ID'])

NameError: name 'record' is not defined

In [ ]:
'''
Loại bỏ 3 thuộc tính có chỉ số corr thấp nhất Xylene(0.07), Benzene(0.21), SO2(0.25) và các dữ liệu không phải dạng số
'''
df = df.drop(columns=['City', 'Date', 'AQI_Bucket', 'Xylene', 'Benzene', 'SO2'])
df.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,O3,Toluene,AQI
0,NaN,NaN,0.92,18.22,17.15,NaN,0.92,133.36,0.02,NaN
1,NaN,NaN,0.97,15.69,16.46,NaN,0.97,34.06,5.50,NaN
2,NaN,NaN,17.40,19.30,29.70,NaN,17.40,30.70,16.40,NaN
3,NaN,NaN,1.70,18.48,17.97,NaN,1.70,36.08,10.14,NaN
4,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.31,18.89,NaN


In [ ]:
# Bỏ các giá trị trống
df = df.dropna()
df.shape

(10960, 10)

### DATABASE CONNECTION

In [ ]:
# Tạo database cho dữ liệu sau khi xử lý
try:
    connection = mysql.connect(host='localhost', database='air_quality', user='root', password='cuong#Super2001')
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute(
            'CREATE TABLE if not exists preprocessed_air_quality (ID int NOT NULL AUTO_INCREMENT PRIMARY KEY,`PM2.5` double,`PM10` double,`NO` double,`NO2` double,NOx double,`NH3` double,`CO` double,O3 double,Toluene double,AQI double)')
        print('New table was created')
        connection.commit()
        cursor.close()
except Error as e:
    print('Error', e)

New table was created


In [ ]:
# Import records of corr_matrix into database
try:
    connection = mysql.connect(host='localhost', database='air_quality', user='root', password='cuong#Super2001')
    if connection.is_connected():
        cursor = connection.cursor()
        for i, row in df.iterrows():
            sql = "INSERT INTO preprocessed_air_quality(`PM2.5`,`PM10`,`NO`,`NO2`,NOx,`NH3`,`CO`,O3,Toluene,AQI) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            # print('Inserted record')
        connection.commit()
        cursor.close()
except Error as e:
    print('Error', e)

In [ ]:
# Nếu đã tồn tại, có thể xóa để tạo một database khác

# try:
#     connection = mysql.connect(host='localhost', database='air_quality', user='root', password='cuong#Super2001')
#     if connection.is_connected():
#         cursor = connection.cursor()
#         tbl_name = "preprocessed_air_quality"
#         sql = "drop table if exists " + tbl_name
#         cursor.execute(sql)
#         print('Table was dropped')
#         connection.commit()
#         cursor.close()
# except Error as e:
#     print('Error', e)


Table was dropped


# Corr

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('city_day.csv')
df = df.dropna()
df.shape

In [ ]:
def get_full_corr(dataset): # In ra ma trận hệ số corr với tất cả các hàng cột của data (>< 1m10s)
    # remove object type
    drop_col = []
    for i in dataset.columns:
        if dataset[i].dtypes == object:
            drop_col.append(i)
    dataset = dataset.drop(columns = drop_col)

    list_cov_x_y = []
    len_col = len(dataset.columns)
    len_row = len(dataset)
    labels = list(dataset.columns)
    print(labels)
    #COV(x, y)
    for i in range(0,len_col):
        cov_feat = []
        i_mean = dataset.iloc[:,i].mean()
        for k in range(0, len_col):
            cov_x_y = 0.0
            k_mean = dataset.iloc[:,k].mean()
            for j in range(0,len_row):
                cov_x_y += (dataset.iloc[j,i]-i_mean)*(dataset.iloc[j,k]-k_mean)
            cov_x_y /= len_row
            cov_feat.append(cov_x_y)
        list_cov_x_y.append(cov_feat)
    print(len(list_cov_x_y))
    #COV(x)
    list_cov_x = []
    for i in range(0,len_col):
        i_mean = dataset.iloc[:,i].mean()
        cov_x = 0.0
        for j in range(0,len_row):
            cov_x += (dataset.iloc[j,i]-i_mean)**2
        cov_x /= (len_row-1)
        list_cov_x.append(cov_x**0.5)
    #COV(y)
    list_cov_y = []
    for i in range(0,len_col):
        cov_y_feat = []
        for k in range(0, len_col):
            cov_y = 0.0
            k_mean = dataset.iloc[:,k].mean()
            for j in range(0,len_row):
                cov_y += (dataset.iloc[j,k]-k_mean)**2
            cov_y /= (len_row-1)
            cov_y_feat.append(cov_y**0.5)
        list_cov_y.append(cov_y_feat)
    #CORR(x,y)
    list_corr = []
    for i in range(0,len_col):
        tmp_list = []
        for j in range(0, len_col):
            tmp_list.append(list_cov_x_y[i][j]/(list_cov_x[i]*list_cov_y[i][j]))
        list_corr.append(tmp_list)
    #CREATE CORR DATAFRAME
    list_corr = pd.DataFrame(list_corr, columns=labels)
    labels = pd.Index(labels)
    list_corr = list_corr.set_index(labels)
    return list_corr

In [ ]:
corr_matrix = get_full_corr(df)
plt.figure(figsize=(10, 10))
plt.title('Correlation Heatmap of Dataset')
a = sns.heatmap(corr_matrix, square=True, annot=True, fmt='.2f', linecolor='black')
a.set_xticklabels(a.get_xticklabels(), rotation=30)
a.set_yticklabels(a.get_yticklabels(), rotation=30)           
plt.show()  

In [ ]:
def get_corr(df): # In ra ma trận hệ số corr với duy nhất cột nhãn của data (>< 6.6s)
    # remove object type
    drop_col = []
    for i in df.columns:
        if df[i].dtypes == object:
            drop_col.append(i)
    df = df.drop(columns = drop_col)

    list_cov_x_y = []
    len_col = len(df.columns)
    len_row = len(df)
    labels = list(df.columns)
    print(labels)
    #COV(x, y)
    list_cov_x_y = []
    len_col = len(df.columns)
    len_row = len(df)
    print(len_col, len_row)
    y_mean = df.iloc[:,len_col-1].mean()
    for i in range(0,len_col):
        cov_x_y = 0.0
        x_mean = df.iloc[:,i].mean()
        for j in range(0,len_row):
            cov_x_y += (df.iloc[j,i]-x_mean)*(df.iloc[j,len_col-1]-y_mean)
        cov_x_y /= len_row
        list_cov_x_y.append(cov_x_y)
    #COV(x)
    list_cov_x = []
    for i in range(0,len_col):
        cov_x = 0.0
        x_mean = df.iloc[:,i].mean()
        for j in range(0,len_row):
            cov_x += (df.iloc[j,i]-x_mean)**2
        cov_x /= (len_row-1)
        list_cov_x.append(cov_x**0.5)
    #COV(y)
    list_cov_y = []
    y_mean = df.iloc[:,len_col-1].mean()
    for i in range(0,len_col):
        cov_y = 0.0
        for j in range(0,len_row):
            cov_y += (df.iloc[j,len_col-1]-y_mean)**2
        cov_y /= (len_row-1)
        list_cov_y.append(cov_y**0.5)
    #CORR(x,y)
    list_corr = []
    for i in range(0,len_col):
        list_corr.append(list_cov_x_y[i]/(list_cov_x[i]*list_cov_y[i]))
    #CREATE CORR DATAFRAME
    list_corr = pd.DataFrame(list_corr, columns=['Y'])
    labels = pd.Index(labels)
    list_corr = list_corr.set_index(labels)
    return list_corr

In [ ]:
corr_matrix = get_corr(df)
plt.figure(figsize=(10, 10))
plt.title('Correlation Heatmap of Dataset')
a = sns.heatmap(corr_matrix, square=True, annot=True, fmt='.2f', linecolor='black')
a.set_xticklabels(a.get_xticklabels(), rotation=30)
a.set_yticklabels(a.get_yticklabels(), rotation=30)           
plt.show()  

In [ ]:
def train_test_split(features, label_name, test_size, random_state):
    shuffle_feature_df = features.sample(frac = 1,random_state=random_state)
    test_size = int(test_size*len(features))
    X_train = shuffle_feature_df[test_size:]
    X_test = shuffle_feature_df[:test_size]
    y_train = X_train[label_name]
    y_test = X_test[label_name]
    X_train = X_train.drop(columns=[label_name])
    X_test = X_test.drop(columns=[label_name])
    return X_train, X_test, y_train, y_test

In [ ]:
X = df.drop(columns=['City','Date','AQI_Bucket'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,'AQI',test_size=0.95,random_state=1)

In [ ]:
print(X_train.shape,' ',X_test.shape,' ',df.shape)

In [ ]:
X_train.head()

# Decision Tree Regressor

# Random Forest

# Lasso Regression

# Linear Regression

# KNeighbors Regressor